In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [3]:
#Cargo los sets
X_train = pd.read_csv('data/X_train_features.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_train = pd.read_csv('data/y_train.csv',header=None,squeeze=True,index_col=0)
X_test = pd.read_csv('data/X_test_features.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_test = pd.read_csv('data/y_test.csv',header=None,squeeze=True,index_col=0)

In [4]:
features_n = pd.read_csv('data/features_importancia_tercera_etapa.csv',index_col=0)\
             .head(50).index.tolist()
features_n[0:5]

['metroscubiertos',
 'banos',
 'metrostotales',
 'provincia__Distrito Federal',
 'idzona']

In [5]:
X_train_n = X_train[features_n]
X_test_n = X_test[features_n]

In [6]:
features_numericos = X_train_n.dtypes[X_train.dtypes != "object"].index
len(features_numericos)
#son todas numericas obviamente

50

In [7]:
scaler = StandardScaler()

scaled = scaler.fit_transform(X_train_n[features_numericos])
for i, col in enumerate(features_numericos):
    X_train_n[col] = scaled[:, i]

scaled = scaler.fit_transform(X_test_n[features_numericos])
for i, col in enumerate(features_numericos):
    X_test_n[col] = scaled[:, i]

/home/franco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/franco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
def RMSE(actual, pred):
    return np.sqrt(mean_squared_error(actual, pred))

In [10]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [11]:
def get_best_score(grid):
    
    best_score = np.sqrt(-grid.best_score_)
    print(best_score)    
    print(grid.best_params_)
    print(grid.best_estimator_)
    
    return best_score

In [13]:
param_grid = {'n_neighbors' : [10] ,    
              'weights' : ['distance'] ,
              'algorithm' : ['auto']}

nr_cv = 2
score_calc = 'neg_mean_squared_error'

grid_knn = GridSearchCV(KNeighborsRegressor(), param_grid, cv=nr_cv, refit=True, verbose=1, scoring = score_calc, n_jobs=4)
grid_knn.fit(X_train_n, y_train_log)

sc_knn = get_best_score(grid_knn)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed: 13.3min finished


0.41664558477484653
{'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'distance'}
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                    weights='distance')


In [ ]:
y_test_pred_knn = grid_knn.predict(X_test_n)
y_train_pred_knn = grid_knn.predict(X_train_n)

In [ ]:
print("RMSE Train:",RMSE(y_train_pred_knn,y_train_log))
print("RMSE Test:",RMSE(y_test_pred_knn,y_test_log))